<a href="https://colab.research.google.com/github/nrjcs/swym/blob/master/swym_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

cifar10 dataset:

Dataset of 50,000 32x32 color training images, labeled over 10/100 categories, and 10,000 test images.

from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()




In [0]:
#import
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np
#define parameters
batch_size=32
nb_epochs=2
kernel_size=3
pool_size=2
conv_depth_1=32
conv_depth_2=64
drop_prob_1=.5
drop_prob_2=.5
hidden_layer_size=512

#loading data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print (X_train.shape)

#preprocessing
num_train, height, width, depth = X_train.shape
num_classes = np.unique(y_train).shape[0]
print (num_classes)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_test)
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)
print (Y_train.shape)
print (Y_test.shape)

inp = Input(shape=(height, width, depth))
print (inp.shape)

#define model
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
flat = Flatten()(drop_2)
hidden = Dense(hidden_layer_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

#create....another way
model = Model(inputs=inp, outputs=out)

# compile
print ("compile now..")
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print("compilation done")

# train
model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epochs,verbose=0, validation_split=0.1)


# evaluate
model.evaluate(X_test, Y_test, verbose=1)


# save
model.save('cifartest_1.h5')



Reproduced from https://keras.io/getting-started/faq/#savingloading-only-a-models-weights



---



How can I save a Keras model?

Saving/loading whole models (architecture + weights + optimizer state)

>It is not recommended to use pickle or cPickle to save a Keras model.

>You can use model.save(filepath) to save a Keras model into a single HDF5 file which will contain:

>>the architecture of the model, allowing to re-create the model

>>the weights of the model

>>the training configuration (loss, optimizer)

>>the state of the optimizer, allowing to resume training exactly where you left off.

>You can then use keras.models.load_model(filepath) to reinstantiate your model. load_model will also take care of compiling the model using the saved training configuration (unless the model was never compiled in the first place).

Saving/loading only a model's architecture

Save the weights of a model

Load the weights into a different architecture